In [1]:
import pandas as pd
import numpy as np
import os
import random
import datetime

path = os.getcwd()
name = pd.read_csv(path + '/name_abb.csv')
name_dict = dict(zip(name['Franchise'],name['Abbreviation/Acronym']))


In [349]:
# import the lineup csv
temp = pd.read_csv(path + '/lineups_adv_source_stats/lineup_adv_stats_2nd_attempt.csv')
sche = pd.read_csv(path+'/total_schedule_4th_attempt.csv')


In [350]:
sche.columns

Index(['Date', 'Start (ET)', 'Visitor/Neutral', 'PTS', 'Home/Neutral', 'PTS.1',
       'Unnamed: 6', 'Unnamed: 7', 'Attend.', 'Notes'],
      dtype='object')

In [351]:
sche_home_away_date.columns

Index(['Visitor/Neutral', 'Home/Neutral', 'TEAM', 'GAMEDAY', 'SEASON',
       'AWAYSCORE', 'HOMESCORE', 'GAMEDAY_FMT', 'SEASON_FMT', 'AWAY',
       'TEAM_DECHANGED', 'AWAY_DECHANGED'],
      dtype='object')

Two source tables:
    - temp: contains lineup data
    - sche: contains schedule

In [366]:
# there is no away team, so I am going to find one.
def sche_cleanup(sche):
    # format date into datetime
    sche['Date_c'] = sche['Date'].str[4:]
    datelist = []
    for date in sche['Date_c']:
        datelist.append(datetime.datetime.strptime(date, '%b %d %Y'))
    sche['Date_fmt'] = datelist
    
    # format Home Abbreviation
    sche['Home_abb'] = [name_dict[n] for n in sche['Home/Neutral']]
   
    # format season
    season = []
    for gameday in sche['Date_fmt']:
        if datetime.date(2006,10,1) <= gameday.date() <= datetime.date(2007,7,1):
            season.append('2006-07')
        elif datetime.date(2007,10,1) <= gameday.date() <= datetime.date(2008,7,1):
            season.append('2007-08')
        elif datetime.date(2008,10,1) <= gameday.date() <= datetime.date(2009,7,1):
            season.append('2008-09')
        elif datetime.date(2009,10,1) <= gameday.date() <= datetime.date(2010,7,1):
            season.append('2009-10')
        elif datetime.date(2010,10,1) <= gameday.date() <= datetime.date(2011,7,1):
            season.append('2010-11')
        elif datetime.date(2011,10,1) <= gameday.date() <= datetime.date(2012,7,1):
            season.append('2011-12')        
        elif datetime.date(2012,10,1) <= gameday.date() <= datetime.date(2013,7,1):
            season.append('2012-13')       
        elif datetime.date(2013,10,1) <= gameday.date() <= datetime.date(2014,7,1):
            season.append('2013-14')    
        elif datetime.date(2014,10,1) <= gameday.date() <= datetime.date(2015,7,1):
            season.append('2014-15')
        elif datetime.date(2015,10,1) <= gameday.date() <= datetime.date(2016,7,1):
            season.append('2015-16')     
        elif datetime.date(2016,10,1) <= gameday.date() <= datetime.date(2017,7,1):
            season.append('2016-17')  
        elif datetime.date(2017,10,1) <= gameday.date() <= datetime.date(2018,7,1):
            season.append('2017-18')
        elif datetime.date(2018,10,1) <= gameday.date() <= datetime.date(2019,7,1):
            season.append('2018-19')
    sche['Season'] = season
    # format regular season
        # mark the regular season
    regularseason = []
    for gameday in sche['Date_fmt']:
        if datetime.date(2006,10,31) <= gameday.date() <= datetime.date(2007,4,18):
            regularseason.append(1)
        elif datetime.date(2007,10,30) <= gameday.date() <= datetime.date(2008,4,16):
            regularseason.append(1)
        elif datetime.date(2008,10,28) <= gameday.date() <= datetime.date(2009,4,16):
            regularseason.append(1)
        elif datetime.date(2009,10,27) <= gameday.date() <= datetime.date(2010,4,14):
            regularseason.append(1)
        elif datetime.date(2010,10,26) <= gameday.date() <= datetime.date(2011,4,13):
            regularseason.append(1)
        elif datetime.date(2011,12,25) <= gameday.date() <= datetime.date(2012,4,26):
            regularseason.append(1)        
        elif datetime.date(2012,10,30) <= gameday.date() <= datetime.date(2013,4,17):
            regularseason.append(1)    
        elif datetime.date(2013,10,29) <= gameday.date() <= datetime.date(2014,4,16):
            regularseason.append(1)  
        elif datetime.date(2014,10,28) <= gameday.date() <= datetime.date(2015,4,15):
            regularseason.append(1)
        elif datetime.date(2015,10,27) <= gameday.date() <= datetime.date(2016,4,13):
            regularseason.append(1)   
        elif datetime.date(2016,10,25) <= gameday.date() <= datetime.date(2017,4,12):
            regularseason.append(1)
        elif datetime.date(2017,10,17) <= gameday.date() <= datetime.date(2018,4,11):
            regularseason.append(1)
        elif datetime.date(2018,10,16) <= gameday.date() <= datetime.date(2019,4,10):
            regularseason.append(1)
        else:
            regularseason.append(0)
    sche['RegularSeason'] = regularseason
    
    # only regular season
    sche = sche.loc[sche['RegularSeason'] == 1]
    
    # thinning!
    sche_home_away_date = sche[['Visitor/Neutral','Home/Neutral',
                                'Home_abb',
                                'Date_fmt',
                                'Season','PTS','PTS.1']]

    sche_home_away_date.columns = ['Visitor/Neutral','Home/Neutral',
                                   'TEAM','GAMEDAY',
                                   'SEASON','AWAYSCORE','HOMESCORE']

    # format the date into datetime stamp
    sche_home_away_date['GAMEDAY_FMT'] = pd.to_datetime(sche_home_away_date.GAMEDAY)
    sche_home_away_date['SEASON_FMT'] = [str(season)[0:4] for season in  sche_home_away_date['SEASON']]
    # find the abbrevation for away team
    sche_home_away_date['AWAY'] = [name_dict[n] for n in sche_home_away_date['Visitor/Neutral']]
        
    sche_home_away_date['TEAM_DECHANGED'] = sche_home_away_date['TEAM']
    sche_home_away_date['AWAY_DECHANGED'] = sche_home_away_date['AWAY']

    sche_home_away_date['TEAM_DECHANGED'] = sche_home_away_date.TEAM_DECHANGED.replace({'NOH':'NOP',
                                               'SEA':'OKC',
                                               'NJN':'BKN'})
    sche_home_away_date['AWAY_DECHANGED'] = sche_home_away_date.AWAY_DECHANGED.replace({'NOH':'NOP',
                                               'SEA':'OKC',
                                               'NJN':'BKN'})

    # remove duplicate
    sche_home_away_date = sche_home_away_date.drop_duplicates()
    return sche_home_away_date

In [353]:
def lineup_cleanup(temp):
    # add the season in temp
    temp['GAMEDAY_FMT'] = pd.to_datetime(temp['GAMEDAY'])
    temp['SEASON'] = [gameday.year for gameday in temp['GAMEDAY_FMT']]

    # dechanged the team names
    temp['TEAM_DECHANGED'] = temp['TEAM']
    temp['TEAM_DECHANGED'] = temp.TEAM_DECHANGED.replace({'NOH':'NOP',
                                           'SEA':'OKC',
                                           'NJN':'BKN'})
    return temp

In [367]:
temp = lineup_cleanup(temp)
sche_home_away_date = sche_cleanup(sche)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

# Features to calculate attempt1

This notebook aims to calculate the advantage part of the featues.
- Offensive Advantages
    - "OFFRTG of A - DEFRTG of B" in the most recent records
        - OFFRTG of A is calculated as the weighted average of the top 10 lineups
        - DEFRTG of B is calculated as the weighted average of the top 10 lineups 

Let us try to start from a small instance to get a feel.

Randomly picked a game.
 - 2018.12.30 MIN vs. MIA
 - Previous n Games

In [56]:
# find all previous games of two team
# including their last season
def findpreviousgames(home, away, pred_day, season, include_last = True):
    prev_season = str(int(season) -1)
    all_last_season_home_games = sche_home_away_date.loc[(sche_home_away_date['SEASON_FMT'] == prev_season) & 
                                                 (sche_home_away_date['GAMEDAY_FMT'] < pred_day) &
                                                 ((sche_home_away_date['TEAM'] == home) & 
                                                 (sche_home_away_date['AWAY'] == away))]
    all_last_season_away_games = sche_home_away_date.loc[(sche_home_away_date['SEASON_FMT'] == prev_season) & 
                                                 (sche_home_away_date['GAMEDAY_FMT'] < pred_day) &
                                                 ((sche_home_away_date['TEAM'] == away) & 
                                                 (sche_home_away_date['AWAY'] == home))]
    all_this_season_home_games = sche_home_away_date.loc[(sche_home_away_date['SEASON_FMT'] == season) & 
                                                 (sche_home_away_date['GAMEDAY_FMT'] < pred_day) &
                                                 ((sche_home_away_date['TEAM'] == home) & 
                                                 (sche_home_away_date['AWAY'] == away))]
    all_this_season_away_games = sche_home_away_date.loc[(sche_home_away_date['SEASON_FMT'] == season) & 
                                                 (sche_home_away_date['GAMEDAY_FMT'] < pred_day) &
                                                 ((sche_home_away_date['TEAM'] == away) & 
                                                 (sche_home_away_date['AWAY'] == home))]
    if include_last:
        return pd.DataFrame(pd.concat([all_last_season_home_games,
                                    all_last_season_away_games,
                                    all_this_season_home_games,
                                    all_this_season_home_games]))
    else:
        return pd.DataFrame(pd.concat([all_this_season_home_games,
                                    all_this_away_games]))

In [57]:
# testing GSW	NOP	2013-12-17
findpreviousgames(home = 'NOP', away = 'CHA', pred_day = '2013-11-02', season = '2013')

,Visitor/Neutral,Home/Neutral,TEAM,GAMEDAY,SEASON,AWAYSCORE,HOMESCORE,GAMEDAY_FMT,AWAY,SEASON_FMT


In [122]:
# find lineups data
def find_lineups_stats(team,gameday):
    stats = temp.loc[(temp['TEAM'] == team) & (temp['GAMEDAY_FMT'] == gameday)]
    return stats

# calculate the relevant features: weighted average of their feature
# weighted by time played in the game
def calculate_weighted_feature(season,lineup_stats,
                               feature,
                               max_lineup_considered = 10, 
                               time_disc = 0.90):
    try:
        total_time = np.sum(lineup_stats['MIN'])
        weighted_feature = 0

        for index, lineup in lineup_stats.iterrows():

            # previous data is discounted
            if int(lineup['SEASON']) < int(season):
                weighted_feature += \
                time_disc * (((lineup['MIN']/total_time) * lineup[feature]))
            else:
                weighted_feature += \
                (((lineup['MIN']/total_time) * lineup[feature]))
            if index > 10:
                break
        
        return weighted_feature
            
    except:
        print('Error')
        print(lineup_stats.head())
    

In [59]:
# testing:
bos181016 = find_lineups_stats('BOS','2018-10-16')
# print(calculate_weighted_feature(lineup_stats=bos181016,feature = 'OFFRTG'))
# print(calculate_weighted_feature(lineup_stats=bos181016,feature = 'DEFRTG'))

In [60]:
# calculate the feature for game 2018.12.30 MIN vs. MIA
previous_games_test = findpreviousgames(home = 'MIA', away = 'MIN', pred_day = '2018-12-30', season = '2018')
# find the relevant lineup stats for two teams
def find_lineup_records_for_two_teams(games, home, away):
    team1_stats = pd.DataFrame()
    team2_stats = pd.DataFrame()
    for index, game in games.iterrows():
        # we don't know who is home
        team1 = game['TEAM']
        team2 = game['AWAY']

        if index == 1:
            team1_stats = find_lineups_stats(home,game['GAMEDAY'])
            team2_stats = find_lineups_stats(away,game['GAMEDAY'])
        else:
            team1_stats = pd.concat([team1_stats,find_lineups_stats(home,game['GAMEDAY'])])
            team2_stats = pd.concat([team2_stats,find_lineups_stats(away,game['GAMEDAY'])])

    return (team1_stats,team2_stats)

In [61]:
# testing
result0 = find_lineup_records_for_two_teams(previous_games_test)[0]
result1 = find_lineup_records_for_two_teams(previous_games_test)[1]
print('Weighted OFF of MIA:', calculate_weighted_feature(pred_season = '2018', lineup_stats=result0,feature = 'OFFRTG'))
print('Weighted OFF of MIN:', calculate_weighted_feature(pred_season = '2018', lineup_stats=result1,feature = 'OFFRTG'))

TypeError: find_lineup_records_for_two_teams() missing 2 required positional arguments: 'home' and 'away'

In [63]:
previous_games_test = findpreviousgames(home = 'MIA', away = 'MIN', pred_day = '2018-12-30', season = '2018')
previous_games_test

,Visitor/Neutral,Home/Neutral,TEAM,GAMEDAY,SEASON,GAMEDAY_FMT,AWAY,SEASON_FMT
13971,Minnesota Timberwolves,Miami Heat,MIA,2017-10-30,2017-18,2017-10-30,MIN,2017
13924,Miami Heat,Minnesota Timberwolves,MIN,2017-11-24,2017-18,2017-11-24,MIA,2017


# Next Step:
- calculate the feature for all games records
- The final table is called 'targ_data'

In [10]:
targ_data = sche_home_away_date[['TEAM','AWAY',
                                'GAMEDAY_FMT','SEASON_FMT',
                                'AWAYSCORE','HOMESCORE']]
# construct a target variable
targ_data['HOMEWIN'] = [int(int(game['AWAYSCORE']) < int(game['HOMESCORE'])) \
                        for index, game in targ_data.iterrows()]

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


For each game, we should:
- Find all its previous games
- Calculate the weighted average of the features
    - nextstep is to make the calculation of a list of features possible

In [31]:
# testing on five games
targ_data_test = targ_data.iloc[937:938]
HOME_FEATURE1,AWAY_FEATURE1,HOME_FEATURE2,AWAY_FEATURE2 = ([] for _ in range(4))
homestorage = dict(zip(list(range(1,3)),[HOME_FEATURE1,HOME_FEATURE2]))
awaystorage = dict(zip(list(range(1,3)),[AWAY_FEATURE1,AWAY_FEATURE2]))
for index, game in targ_data_test.iterrows():

    # first finds all previous games
    home = game['TEAM']
    away = game['AWAY']
    gameday = game['GAMEDAY_FMT']
    season = game['SEASON_FMT']
    feature = ['OFFRTG','DEFRTG']
    home_feature = 0
    away_feature = 0
    previous_games_now = findpreviousgames(home = home, away = away,
                                      pred_day = gameday,
                                      season = season)
    print(previous_games_now.head())
    # then finds the relevant lineup stats
    homestats, awaystats = find_lineup_records_for_two_teams(previous_games_now,home = home, away = away)
    
    # then calculate the weighted features
    for i in range(len(feature)):
        home_feature = calculate_weighted_feature(season = season, 
                                                   lineup_stats= homestats,
                                                   feature = feature[i])

        away_feature = calculate_weighted_feature(season = season, 
                                                   lineup_stats= awaystats,
                                                   feature = feature[i])
        # store it somewhere
        homestorage[i+1].append(home_feature)
        awaystorage[i+1].append(away_feature)


        Visitor/Neutral     Home/Neutral TEAM     GAMEDAY   SEASON  AWAYSCORE  \
967  Los Angeles Lakers  Houston Rockets  HOU  2007-11-14  2007-08         93   
967  Los Angeles Lakers  Houston Rockets  HOU  2007-11-14  2007-08         93   

     HOMESCORE GAMEDAY_FMT AWAY SEASON_FMT  
967         90  2007-11-14  LAL       2007  
967         90  2007-11-14  LAL       2007  


In [17]:
sche.to_csv('schedule_with_advantage.csv')

In [3]:
temp.columns

Index(['lineup', 'TEAM', 'GP', 'MIN', 'OFFRTG', 'DEFRTG', 'NETRTG', 'AST%',
       'AST/TO', 'AST RATIO', 'OREB%', 'DREB%', 'REB%', 'TO RATIO', 'EFG%',
       'TS%', 'PACE', 'PIE', 'GAMEDAY'],
      dtype='object')

In [4]:
namelist = ['AST%',
       'AST/TO', 'AST RATIO', 'OREB%', 'DREB%', 'REB%', 'TO RATIO', 'EFG%',
       'TS%', 'PACE', 'PIE']

# Tackle The Missing Values
 - Wrong TEAM ID that leads to lack of lineup data
     - SAS AND WAS
     - SOLUTION:
         - We should make sure the date is formatted correctly
 - Wrong TEAM NAME that leads to failure of searching lineup data
     - NOP
     - OKC
     - BKN

In [115]:
# sort by team and gameday
temp.sort_values(by = ['TEAM','GAMEDAY_FMT','SEASON'])
temp = temp.drop(['GAMDAY','GAMDAY_FMT'], axis = 1)

In [116]:
# Wrong Team ID
# check whether second-updated lineup source table contains SAS and WAS data
temp.loc[temp['TEAM'] == 'SAS']['GAMEDAY_FMT'].unique()# we have Spurs linesup data

array(['2007-11-13T00:00:00.000000000', '2007-11-15T00:00:00.000000000',
       '2007-11-25T00:00:00.000000000', '2007-12-02T00:00:00.000000000',
       '2007-12-17T00:00:00.000000000', '2007-12-30T00:00:00.000000000',
       '2008-01-03T00:00:00.000000000', '2008-01-07T00:00:00.000000000',
       '2008-01-10T00:00:00.000000000', '2008-01-17T00:00:00.000000000',
       '2008-01-19T00:00:00.000000000', '2008-01-24T00:00:00.000000000',
       '2008-01-26T00:00:00.000000000', '2008-01-28T00:00:00.000000000',
       '2008-01-31T00:00:00.000000000', '2008-02-21T00:00:00.000000000',
       '2008-02-28T00:00:00.000000000', '2008-03-01T00:00:00.000000000',
       '2008-03-04T00:00:00.000000000', '2008-03-06T00:00:00.000000000',
       '2008-03-09T00:00:00.000000000', '2008-03-10T00:00:00.000000000',
       '2008-03-12T00:00:00.000000000', '2008-03-14T00:00:00.000000000',
       '2008-03-20T00:00:00.000000000', '2008-04-01T00:00:00.000000000',
       '2016-01-08T00:00:00.000000000', '2010-01-02

In [117]:
temp.loc[temp['TEAM'] == 'WAS'] # we have WAS lineups data

,Unnamed: 0,lineup,TEAM,GP,MIN,OFFRTG,DEFRTG,NETRTG,AST%,AST/TO,...,DREB%,REB%,TO RATIO,EFG%,TS%,PACE,PIE,GAMEDAY,SEASON,GAMEDAY_FMT
93,93,"A. Daniels, .A. Jamison, .D. Stevenson, .B. Ha...",WAS,1,11,134.8,72.7,62.1,50.0,2.00,...,80.0,62.5,0.1,64.3,66.8,96.14,89.9,11-20-2007,2007,2007-11-20
94,94,"R. Mason Jr., .D. Songaila, .A. Blatche, .N. Y...",WAS,1,10,60.0,138.1,-78.1,50.0,0.60,...,50.0,38.1,0.3,35.3,35.3,98.96,13.9,11-20-2007,2007,2007-11-20
95,95,"A. Daniels, .A. Jamison, .D. Stevenson, .C. Bu...",WAS,1,5,122.2,60.0,62.2,50.0,0.00,...,66.7,53.3,0.0,55.6,55.7,86.66,84.4,11-20-2007,2007,2007-11-20
96,96,"A. Daniels, .A. Jamison, .D. Stevenson, .D. So...",WAS,1,4,128.6,116.7,11.9,0.0,0.00,...,100.0,71.4,0.0,50.0,53.3,81.39,58.6,11-20-2007,2007,2007-11-20
97,97,"A. Daniels, .A. Jamison, .D. Songaila, .N. You...",WAS,1,3,120.0,150.0,-30.0,100.0,2.00,...,0.0,0.0,0.2,66.7,77.3,91.03,43.5,11-20-2007,2007,2007-11-20
98,98,"A. Daniels, .D. Stevenson, .B. Haywood, .C. Bu...",WAS,1,3,150.0,120.0,30.0,100.0,0.00,...,66.7,40.0,0.0,66.7,69.9,82.07,66.7,11-20-2007,2007,2007-11-20
99,99,"A. Jamison, .D. Stevenson, .D. Songaila, .A. B...",WAS,1,3,150.0,100.0,50.0,50.0,2.00,...,66.7,60.0,0.2,80.0,76.5,90.00,80.0,11-20-2007,2007,2007-11-20
100,100,"A. Daniels, .A. Jamison, .C. Butler, .R. Mason...",WAS,1,2,125.0,100.0,25.0,100.0,2.00,...,0.0,50.0,0.3,83.3,83.3,108.68,75.0,11-20-2007,2007,2007-11-20
101,101,"A. Jamison, .C. Butler, .R. Mason Jr., .A. Bla...",WAS,1,2,140.0,200.0,-60.0,100.0,3.00,...,100.0,50.0,0.2,75.0,78.8,115.71,47.1,11-20-2007,2007,2007-11-20
102,102,"A. Daniels, .A. Jamison, .C. Butler, .D. Songa...",WAS,1,1,200.0,150.0,50.0,100.0,0.00,...,0.0,0.0,0.0,100.0,106.4,81.13,100.0,11-20-2007,2007,2007-11-20


In [282]:
# check whether the calculation methods work
# find Spurs and WAS games

spurs_schedule_test = sche_home_away_date.loc[(sche_home_away_date.AWAY == 'SAS') & 
                       (sche_home_away_date.SEASON_FMT == '2013')]
spurs_schedule_test.shape # only home games

(51, 12)

In [120]:
# WAS Games
WAS_schedule_test = sche_home_away_date.loc[(sche_home_away_date.AWAY == 'WAS') & 
                       (sche_home_away_date.SEASON_FMT == '2013')]
WAS_schedule_test.shape # only home games
WAS_schedule_test.head()

,Visitor/Neutral,Home/Neutral,TEAM,GAMEDAY,SEASON,AWAYSCORE,HOMESCORE,GAMEDAY_FMT,AWAY,SEASON_FMT
8688,Washington Wizards,Detroit Pistons,DET,2013-10-30,2013-14,102,113,2013-10-30,WAS,2013
9001,Washington Wizards,Miami Heat,MIA,2013-11-03,2013-14,93,103,2013-11-03,WAS,2013
7951,Washington Wizards,Philadelphia 76ers,PHI,2013-11-06,2013-14,116,102,2013-11-06,WAS,2013
7735,Washington Wizards,Oklahoma City Thunder,OKC,2013-11-10,2013-14,105,106,2013-11-10,WAS,2013
8913,Washington Wizards,Dallas Mavericks,DAL,2013-11-12,2013-14,95,105,2013-11-12,WAS,2013


In [123]:
# test the method
# testing on five games
targ_data_test = WAS_schedule_test[0:10]
HOME_FEATURE1,AWAY_FEATURE1,HOME_FEATURE2,AWAY_FEATURE2 = ([] for _ in range(4))

feature_name = ['HOME_OFFRTG','AWAY_OFFRTG','HOME_DEFRTG','AWAY_DEFRTG']
feature = ['OFFRTG','DEFRTG']
storage = [[] for _ in range(len(feature_name))]
calculated_features_storage = dict(zip(feature_name,storage))

for index, game in targ_data_test.iterrows():
    # first finds all previous games
    home = game['TEAM']
    away = game['AWAY']
    gameday = game['GAMEDAY_FMT']
    season = game['SEASON_FMT']

    home_feature = 0
    away_feature = 0
    previous_games_now = findpreviousgames(home = home, away = away,
                                      pred_day = gameday,
                                      season = season)
    
    # then finds the relevant lineup stats
    homestats, awaystats = find_lineup_records_for_two_teams(previous_games_now,
                                                             home = home,
                                                             away = away)
#     print(home, homestats)
#     print(away, awaystats)
    # then calculate the weighted features
    for feat in feature:
        home_feature = calculate_weighted_feature(season = season, 
                                                   lineup_stats= homestats,
                                                   feature = feat)

        away_feature = calculate_weighted_feature(season = season, 
                                                   lineup_stats= awaystats,
                                                   feature = feat)
        # store it somewhere
        for key in calculated_features_storage.keys():
            if key.endswith("_"+str(feat)) & key.startswith('H'):
                if home_feature == None:
                    calculated_features_storage[key].append('NaN')
                else:
                    calculated_features_storage[key].append(home_feature)

            elif key.endswith("_"+str(feat)) & key.startswith('A'):
                if away_feature == None:
                    calculated_features_storage[key].append('NaN')
                else:
                    calculated_features_storage[key].append(away_feature)
    result = pd.DataFrame(calculated_features_storage)

In [124]:
result

,HOME_OFFRTG,AWAY_OFFRTG,HOME_DEFRTG,AWAY_DEFRTG
0,10.162539,11.790000,6.255233,29.250000
1,11.836649,4.320000,5.654450,11.966786
2,13.473649,0.000000,15.214189,0.000000
3,13.373404,51.031250,11.990426,48.922917
4,10.760870,12.912766,11.277391,20.478723
5,13.402062,23.881224,4.639175,23.093265
6,17.701049,9.142347,14.066434,11.061735
7,6.693750,8.500000,8.071875,6.459184
8,4.129577,16.666667,8.101408,26.791667
9,7.457749,0.000000,10.009319,0.000000


Problem: can't find the lineup stats for team that changed names
    - Problem 1: Didn't find previous games
    - Problem 2: Didn't find relevant lineups data
 

In [128]:
# set up the testing targets

OKC_schedule_test = sche_home_away_date.loc[(sche_home_away_date.TEAM == 'OKC') & 
                       (sche_home_away_date.SEASON_FMT == '2008')]
NOP_schedule_test = sche_home_away_date.loc[(sche_home_away_date.TEAM == 'NOP') & 
                       (sche_home_away_date.SEASON_FMT == '2013')]
BKN_schedule_test = sche_home_away_date.loc[(sche_home_away_date.TEAM == 'BKN') & 
                       (sche_home_away_date.SEASON_FMT == '2012')]

In [130]:
BKN_schedule_test.head()

,Visitor/Neutral,Home/Neutral,TEAM,GAMEDAY,SEASON,AWAYSCORE,HOMESCORE,GAMEDAY_FMT,AWAY,SEASON_FMT
6638,Toronto Raptors,Brooklyn Nets,BKN,2012-11-03,2012-13,100,107,2012-11-03,TOR,2012
6475,Minnesota Timberwolves,Brooklyn Nets,BKN,2012-11-05,2012-13,107,96,2012-11-05,MIN,2012
6858,Orlando Magic,Brooklyn Nets,BKN,2012-11-11,2012-13,74,82,2012-11-11,ORL,2012
6800,Cleveland Cavaliers,Brooklyn Nets,BKN,2012-11-13,2012-13,101,114,2012-11-13,CLE,2012
6481,Boston Celtics,Brooklyn Nets,BKN,2012-11-15,2012-13,97,102,2012-11-15,BOS,2012


In [190]:
# schedule before name change, all exists
BKN_sche_prior_namechange = sche_home_away_date.loc[(sche_home_away_date.TEAM == 'NJN') & 
                       (sche_home_away_date.SEASON_FMT == '2011')]
OKC_sche_prior_namechange = sche_home_away_date.loc[(sche_home_away_date.TEAM == 'SEA') & 
                       (sche_home_away_date.SEASON_FMT == '2007')]
NOP_sche_prior_namechange = sche_home_away_date.loc[(sche_home_away_date.TEAM == 'NOH') & 
                       (sche_home_away_date.SEASON_FMT == '2012')]



In [191]:
# lineups stats before name change; for testing purpose
BKN_lineup_prior_namechange = temp.loc[temp['TEAM'] == 'NJN']
OKC_lineup_prior_namechange = temp.loc[temp['TEAM'] == 'SEA']
NOP_lineup_prior_namechange = temp.loc[temp['TEAM'] == 'NOH']

In [192]:
# name the change team name as TEAM_de_changed
temp.columns

Index(['Unnamed: 0', 'lineup', 'TEAM', 'GP', 'MIN', 'OFFRTG', 'DEFRTG',
       'NETRTG', 'AST%', 'AST/TO', 'AST RATIO', 'OREB%', 'DREB%', 'REB%',
       'TO RATIO', 'EFG%', 'TS%', 'PACE', 'PIE', 'GAMEDAY', 'SEASON',
       'GAMEDAY_FMT', 'TEAM_DECHANGED'],
      dtype='object')

In [193]:
sche_home_away_date.columns

Index(['Visitor/Neutral', 'Home/Neutral', 'TEAM', 'GAMEDAY', 'SEASON',
       'AWAYSCORE', 'HOMESCORE', 'GAMEDAY_FMT', 'AWAY', 'SEASON_FMT',
       'TEAM_DECHANGED', 'AWAY_DECHANGED'],
      dtype='object')

In [174]:
# TEAM_DECHANGED = np.zeros(sche_home_away_date.shape[0])
# AWAY_DECHANGED = np.zeros(sche_home_away_date.shape[0])

sche_home_away_date['TEAM_DECHANGED'] = sche_home_away_date['TEAM']
sche_home_away_date['AWAY_DECHANGED'] = sche_home_away_date['AWAY']

sche_home_away_date['TEAM_DECHANGED'] = sche_home_away_date.TEAM_DECHANGED.replace({'NOH':'NOP',
                                           'SEA':'OKC',
                                           'NJN':'BKN'})
sche_home_away_date['AWAY_DECHANGED'] = sche_home_away_date.AWAY_DECHANGED.replace({'NOH':'NOP',
                                           'SEA':'OKC',
                                           'NJN':'BKN'})

In [178]:
temp.loc[(temp.TEAM_DECHANGED == 'NOP') & (temp.TEAM == 'NOH')]

,Unnamed: 0,lineup,TEAM,GP,MIN,OFFRTG,DEFRTG,NETRTG,AST%,AST/TO,...,REB%,TO RATIO,EFG%,TS%,PACE,PIE,GAMEDAY,SEASON,GAMEDAY_FMT,TEAM_DECHANGED
4125,4125,"J. Posey, .D. West, .E. Okafor, .C. Paul, .B. ...",NOH,1,9,100.0,93.3,6.7,42.9,0.00,...,41.2,0.0,38.9,40.7,81.36,41.3,01-02-2010,2010,2010-01-02,NOP
4126,4126,"P. Stojakovic, .D. Brown, .D. West, .E. Okafor...",NOH,1,9,106.3,111.8,-5.5,42.9,3.00,...,37.5,0.1,46.9,47.9,87.68,33.3,01-02-2010,2010,2010-01-02,NOP
4127,4127,"P. Stojakovic, .D. Songaila, .D. Brown, .E. Ok...",NOH,1,5,120.0,90.9,29.1,50.0,3.00,...,55.6,0.1,66.7,66.7,94.80,69.8,01-02-2010,2010,2010-01-02,NOP
4128,4128,"P. Stojakovic, .D. Songaila, .E. Okafor, .C. P...",NOH,1,5,80.0,155.6,-75.6,33.3,0.00,...,50.0,0.0,29.2,31.1,91.51,7.9,01-02-2010,2010,2010-01-02,NOP
4129,4129,"J. Posey, .D. Brown, .D. West, .C. Paul, .J. W...",NOH,1,4,100.0,88.9,11.1,50.0,2.00,...,44.4,0.1,50.0,50.0,95.63,57.6,01-02-2010,2010,2010-01-02,NOP
4130,4130,"J. Posey, .D. Brown, .D. West, .E. Okafor, .C....",NOH,1,4,175.0,50.0,125.0,60.0,3.00,...,83.3,0.1,78.6,84.1,88.96,140.7,01-02-2010,2010,2010-01-02,NOP
4131,4131,"P. Stojakovic, .D. West, .E. Okafor, .D. Colli...",NOH,1,3,120.0,166.7,-46.7,0.0,0.00,...,25.0,0.0,50.0,61.5,88.99,27.6,01-02-2010,2010,2010-01-02,NOP
4132,4132,"J. Posey, .D. Songaila, .D. West, .C. Paul, .B...",NOH,1,2,120.0,100.0,20.0,0.0,0.00,...,66.7,0.2,0.0,64.7,92.57,80.0,01-02-2010,2010,2010-01-02,NOP
4133,4133,"J. Posey, .D. Songaila, .D. Brown, .E. Okafor,...",NOH,1,1,66.7,66.7,0.0,100.0,0.00,...,66.7,0.0,33.3,33.3,102.86,50.0,01-02-2010,2010,2010-01-02,NOP
4134,4134,"J. Posey, .D. West, .E. Okafor, .D. Collison, ...",NOH,1,1,0.0,0.0,0.0,0.0,0.00,...,50.0,0.0,0.0,0.0,155.68,0.0,01-02-2010,2010,2010-01-02,NOP


In [179]:
sche_home_away_date.loc[(sche_home_away_date.TEAM_DECHANGED == 'NOP') 
                        & (sche_home_away_date.TEAM == 'NOH')]

,Visitor/Neutral,Home/Neutral,TEAM,GAMEDAY,SEASON,AWAYSCORE,HOMESCORE,GAMEDAY_FMT,AWAY,SEASON_FMT,TEAM_DECHANGED,AWAY_DECHANGED
695,Sacramento Kings,New Orleans Hornets,NOH,2007-10-31,2007-08,90,104,2007-10-31,SAC,2007,NOP,SAC
165,Portland Trail Blazers,New Orleans Hornets,NOH,2007-11-02,2007-08,93,113,2007-11-02,POR,2007,NOP,POR
622,San Antonio Spurs,New Orleans Hornets,NOH,2007-11-09,2007-08,97,85,2007-11-09,SAS,2007,NOP,SAS
560,Philadelphia 76ers,New Orleans Hornets,NOH,2007-11-14,2007-08,76,95,2007-11-14,PHI,2007,NOP,PHI
172,Orlando Magic,New Orleans Hornets,NOH,2007-11-19,2007-08,95,88,2007-11-19,ORL,2007,NOP,ORL
681,Indiana Pacers,New Orleans Hornets,NOH,2007-11-21,2007-08,105,93,2007-11-21,IND,2007,NOP,IND
142,Minnesota Timberwolves,New Orleans Hornets,NOH,2007-11-26,2007-08,103,94,2007-11-26,MIN,2007,NOP,MIN
575,Dallas Mavericks,New Orleans Hornets,NOH,2007-12-01,2007-08,108,112,2007-12-01,DAL,2007,NOP,DAL
710,Detroit Pistons,New Orleans Hornets,NOH,2007-12-05,2007-08,91,76,2007-12-05,DET,2007,NOP,DET
125,Memphis Grizzlies,New Orleans Hornets,NOH,2007-12-07,2007-08,116,118,2007-12-07,MEM,2007,NOP,MEM


In [180]:
temp.to_csv('lineup_adv_stats_3rd_attempt.csv')

In [181]:
sche_home_away_date.to_csv('total_schedule_3rd_attempt.csv')

Next step is to fix the functions:

In [311]:
# find all previous games of two team from sche_home_away_date
# including their last season

def findpreviousgames(home, away, pred_day, season, include_last = True):
    prev_season = str(int(season) -1)
    all_last_season_home_games = sche_home_away_date.loc[(sche_home_away_date['SEASON_FMT'] == prev_season) & 
                                                 (sche_home_away_date['GAMEDAY_FMT'] < pred_day) &
                                                 ((sche_home_away_date['TEAM_DECHANGED'] == home) & 
                                                 (sche_home_away_date['AWAY_DECHANGED'] == away))]
    all_last_season_away_games = sche_home_away_date.loc[(sche_home_away_date['SEASON_FMT'] == prev_season) & 
                                                 (sche_home_away_date['GAMEDAY_FMT'] < pred_day) &
                                                 ((sche_home_away_date['TEAM_DECHANGED'] == away) & 
                                                 (sche_home_away_date['AWAY_DECHANGED'] == home))]
    all_this_season_home_games = sche_home_away_date.loc[(sche_home_away_date['SEASON_FMT'] == season) & 
                                                 (sche_home_away_date['GAMEDAY_FMT'] < pred_day) &
                                                 ((sche_home_away_date['TEAM_DECHANGED'] == home) & 
                                                 (sche_home_away_date['AWAY_DECHANGED'] == away))]
    all_this_season_away_games = sche_home_away_date.loc[(sche_home_away_date['SEASON_FMT'] == season) & 
                                                 (sche_home_away_date['GAMEDAY_FMT'] < pred_day) &
                                                 ((sche_home_away_date['TEAM_DECHANGED'] == away) & 
                                                 (sche_home_away_date['AWAY_DECHANGED'] == home))]
    if include_last:
        return pd.DataFrame(pd.concat([all_last_season_home_games,
                                    all_last_season_away_games,
                                    all_this_season_home_games,
                                    all_this_season_home_games]))
    else:
        return pd.DataFrame(pd.concat([all_this_season_home_games,
                                    all_this_away_games]))
# find lineups data from temp
def find_lineups_stats(team,gameday):
    stats = temp.loc[(temp['TEAM_DECHANGED'] == team) & (temp['GAMEDAY_FMT'] == gameday)]
    return stats

# calculate the relevant features: weighted average of their feature
# weighted by time played in the game
# from temp
def calculate_weighted_feature(season,lineup_stats,
                               feature,
                               max_lineup_considered = 10, 
                               time_disc = 0.90):
#     try:
    total_time = np.sum(lineup_stats['MIN'])
    weighted_feature = 0
    
    for index, lineup in lineup_stats.iterrows():
#         print('lineup MIN, ', lineup['MIN'])
#         print('lineup FEATURE, ', lineup[feature])

        # previous data is discounted
        if int(lineup['SEASON']) < int(season):
            weighted_feature += \
            time_disc * np.multiply(float(lineup['MIN']/total_time),float(lineup[feature]))
        else:
            weighted_feature += \
            np.multiply(float(lineup['MIN']/total_time),float(lineup[feature]))
        if index > 10:
            break

    return weighted_feature

#     except:
#     print('Error')
#         print(lineup_stats.head())


# find the relevant lineup stats for two teams
# using schedule
def find_lineup_records_for_two_teams(games, home, away):
    team1_stats = pd.DataFrame()
    team2_stats = pd.DataFrame()
    for index, game in games.iterrows():
        # we don't know who is home
        team1 = game['TEAM_DECHANGED']
        team2 = game['AWAY_DECHANGED']

        if index == 1:
            team1_stats = find_lineups_stats(home,game['GAMEDAY_FMT'])
            team2_stats = find_lineups_stats(away,game['GAMEDAY_FMT'])
        else:
            team1_stats = pd.concat([team1_stats,find_lineups_stats(home,game['GAMEDAY_FMT'])])
            team2_stats = pd.concat([team2_stats,find_lineups_stats(away,game['GAMEDAY_FMT'])])

    return (team1_stats,team2_stats)

# testing on five games
def feature_calculation(targ_games, 
                        features = 
                        ['OFFRTG','DEFRTG', 'AST%','AST/TO', 
                         'AST RATIO', 'OREB%', 'DREB%', 'REB%', 
                         'TO RATIO', 'EFG%','TS%', 'PACE', 'PIE']):
    

    
    # prepare the columns names
    columns_names = []
    for feat in features:
        columns_names.append('HOME_'+str(feat))
        columns_names.append('AWAY_'+str(feat))
        
    storage = [[] for _ in range(len(columns_names))]

    calculated_features_storage = dict(zip(columns_names,storage))
    
    for index, game in targ_games.iterrows():

        # first finds all previous games
        home = game['TEAM_DECHANGED']
        away = game['AWAY_DECHANGED']
        gameday = game['GAMEDAY_FMT']
        season = game['SEASON_FMT']
        home_feature = 0
        away_feature = 0
        previous_games_now = findpreviousgames(home = home, away = away,
                                          pred_day = gameday,
                                          season = season)
#         print(previous_games_now.head())
        # then finds the relevant lineup stats
        homestats, awaystats = find_lineup_records_for_two_teams(previous_games_now,home = home, away = away)

        #         print(homestats,awaystats)
        # then calculate the weighted features
        for feat in features:
#             print(feat)
            home_feature = calculate_weighted_feature(season = season, 
                                                       lineup_stats= homestats,
                                                       feature = feat)

            away_feature = calculate_weighted_feature(season = season, 
                                                       lineup_stats= awaystats,
                                                       feature = feat)
#             print(feat)
#             print(home_feature,away_feature)

            # store it somewhere
            for key in calculated_features_storage.keys():
                if (key.endswith("_"+str(feat))) & (key.startswith('HOME')):
                    if home_feature == None:
                        calculated_features_storage[key].append('NaN')
                    else:
                        calculated_features_storage[key].append(home_feature)

                elif (key.endswith("_"+str(feat))) & (key.startswith('AWAY')):
                    if away_feature == None:
                        calculated_features_storage[key].append('NaN')
                    else:
                        calculated_features_storage[key].append(away_feature)
#             print([len(stor) for stor in calculated_features_storage.values()]) 
    
    # turn it into dataframe
    calculated_feature = pd.DataFrame(calculated_features_storage)
    targ_games.reset_index(drop=True, inplace=True)
    calculated_feature.reset_index(drop=True, inplace=True)
    result = pd.concat([targ_games,calculated_feature], axis = 1)
    return result

In [312]:
# test on OKC 
result_NOP = feature_calculation(targ_games= NOP_sche_prior_namechange)

In [358]:
# test on BKN
result_BKN= feature_calculation(targ_games= BKN_sche_prior_namechange)

In [314]:
# test on SAS
result_SAS =feature_calculation(targ_games= spurs_schedule_test)

In [371]:
result_BKN.columns

Index(['Visitor/Neutral', 'Home/Neutral', 'TEAM', 'GAMEDAY', 'SEASON',
       'AWAYSCORE', 'HOMESCORE', 'GAMEDAY_FMT', 'AWAY', 'SEASON_FMT',
       'TEAM_DECHANGED', 'AWAY_DECHANGED', 'HOME_OFFRTG', 'AWAY_OFFRTG',
       'HOME_DEFRTG', 'AWAY_DEFRTG', 'HOME_AST%', 'AWAY_AST%', 'HOME_AST/TO',
       'AWAY_AST/TO', 'HOME_AST RATIO', 'AWAY_AST RATIO', 'HOME_OREB%',
       'AWAY_OREB%', 'HOME_DREB%', 'AWAY_DREB%', 'HOME_REB%', 'AWAY_REB%',
       'HOME_TO RATIO', 'AWAY_TO RATIO', 'HOME_EFG%', 'AWAY_EFG%', 'HOME_TS%',
       'AWAY_TS%', 'HOME_PACE', 'AWAY_PACE', 'HOME_PIE', 'AWAY_PIE'],
      dtype='object')

All tests passed. 

Next step is to calculate for the entire sample.

# Feature Calculation Attempt2

In [380]:
# get rid of 2006, 2007
entire_after_07 = sche_home_away_date.loc[(sche_home_away_date['SEASON_FMT'] != '2007')
                                      & (sche_home_away_date['SEASON_FMT'] != '2006')]

In [384]:
result_after_07 = feature_calculation(targ_games= entire_after_07)

In [388]:
result_after_07.to_csv('schedule_with_all_advstats_calculated_4th_attempt.csv')

In [2]:
feature_calculated_after_07 = pd.read_csv(path+
                                          '/all_adv_stats_calculated/schedule_with_all_advstats_calculated_4th_attempt.csv')

In [3]:
column = feature_calculated_after_07.columns

In [4]:
column

Index(['Unnamed: 0', 'Visitor/Neutral', 'Home/Neutral', 'TEAM', 'GAMEDAY',
       'SEASON', 'AWAYSCORE', 'HOMESCORE', 'GAMEDAY_FMT', 'SEASON_FMT', 'AWAY',
       'TEAM_DECHANGED', 'AWAY_DECHANGED', 'HOME_OFFRTG', 'AWAY_OFFRTG',
       'HOME_DEFRTG', 'AWAY_DEFRTG', 'HOME_AST%', 'AWAY_AST%', 'HOME_AST/TO',
       'AWAY_AST/TO', 'HOME_AST RATIO', 'AWAY_AST RATIO', 'HOME_OREB%',
       'AWAY_OREB%', 'HOME_DREB%', 'AWAY_DREB%', 'HOME_REB%', 'AWAY_REB%',
       'HOME_TO RATIO', 'AWAY_TO RATIO', 'HOME_EFG%', 'AWAY_EFG%', 'HOME_TS%',
       'AWAY_TS%', 'HOME_PACE', 'AWAY_PACE', 'HOME_PIE', 'AWAY_PIE'],
      dtype='object')

In [11]:
adj_column = []
for col in column:
    if col[0:5] == 'HOME_':
        adj_column.append(col[5:] + "_h")
    elif col[0:5] == 'AWAY_':
        adj_column.append(col[5:] + "_a")
    elif col == 'GAMEDAY_FMT':
        adj_column.append('Date')
    elif col == 'TEAM':
        adj_column.append('Team_h')
    elif col == 'AWAY':
        adj_column.append('Team_a')
    else:
        adj_column.append(col)

In [13]:
feature_calculated_after_07.columns = adj_column

In [14]:
# feature_calculated_after_07.head()

,Unnamed: 0,Visitor/Neutral,Home/Neutral,Team_h,GAMEDAY,SEASON,AWAYSCORE,HOMESCORE,Date,SEASON_FMT,...,TO RATIO_h,TO RATIO_a,EFG%_h,EFG%_a,TS%_h,TS%_a,PACE_h,PACE_a,PIE_h,PIE_a
0,0,Cleveland Cavaliers,Boston Celtics,BOS,2008-10-28,2008-09,85,90,2008-10-28,2008,...,0.020412,0.023782,4.950000,2.283109,5.521546,2.528860,9.588711,7.507306,9.777526,-0.103057
1,1,Milwaukee Bucks,Chicago Bulls,CHI,2008-10-28,2008-09,95,108,2008-10-28,2008,...,0.012062,0.008482,2.261598,5.199267,3.063711,5.250157,6.245629,7.689487,3.455722,4.546178
2,2,Portland Trail Blazers,Los Angeles Lakers,LAL,2008-10-28,2008-09,76,96,2008-10-28,2008,...,0.006186,0.011458,3.439175,5.557292,3.927835,6.416667,5.705567,10.207083,6.408247,6.897917
3,3,Toronto Raptors,Philadelphia 76ers,PHI,2008-10-29,2008-09,95,84,2008-10-29,2008,...,0.000000,0.005567,2.193231,1.809278,2.363077,2.811340,3.224492,5.285876,2.204308,3.028454
4,4,New Jersey Nets,Washington Wizards,WAS,2008-10-29,2008-09,95,85,2008-10-29,2008,...,0.012316,0.013750,6.465789,4.173125,7.069263,4.365625,12.138442,6.316063,11.626105,2.571250


In [15]:
teamid_lookup = pd.read_json(path +'/nba_teamid.json')
TeamID = dict(zip(teamid_lookup['abbreviation'], teamid_lookup['teamId']))

In [17]:
Team_h_id = []
Team_a_id = []

for index,row in feature_calculated_after_07.iterrows():
    Team_h_id.append(TeamID[row['Team_h']])
    Team_a_id.append(TeamID[row['Team_a']])

In [18]:
feature_calculated_after_07['Team_h_id'] = Team_h_id
feature_calculated_after_07['Team_a_id'] = Team_a_id

In [20]:
feature_calculated_after_07.head

<bound method NDFrame.head of        Unnamed: 0         Visitor/Neutral            Home/Neutral Team_h  \
0               0     Cleveland Cavaliers          Boston Celtics    BOS   
1               1         Milwaukee Bucks           Chicago Bulls    CHI   
2               2  Portland Trail Blazers      Los Angeles Lakers    LAL   
3               3         Toronto Raptors      Philadelphia 76ers    PHI   
4               4         New Jersey Nets      Washington Wizards    WAS   
5               5           Atlanta Hawks           Orlando Magic    ORL   
6               6              Miami Heat         New York Knicks    NYK   
7               7            Phoenix Suns       San Antonio Spurs    SAS   
8               8         Milwaukee Bucks   Oklahoma City Thunder    OKC   
9               9        Sacramento Kings  Minnesota Timberwolves    MIN   
10             10          Indiana Pacers         Detroit Pistons    DET   
11             11       Memphis Grizzlies         Houston 

In [21]:
feature_calculated_after_07.to_csv('sche_with_all_advstats_calculated_5th_attempt.csv')